<a href="https://colab.research.google.com/github/DrakeData/House_Price_EDA/blob/main/MSDS422_Assignment1_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 - House Prices: Advanced Regression Techniques
MSDS 422: Practical Machine Learning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

%matplotlib inline
sns.set()

## Ingest
The data set used for this assignment is the [Ames, Iowa Real Estate data set from Kaggle.com](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview). We will be performing Exploratory Data Analysis (EDA) to identify key dependent variables that help determine the sales price of a house.

In [ ]:
# Create file path to the test and trained data set
train_path = os.path.join(os.getcwd(), 'data', 'train.csv')
test_path = os.path.join(os.getcwd(), 'data', 'test.csv')

# Load csv into pandas df
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)